In [69]:
suppressPackageStartupMessages(library(cicero))
suppressPackageStartupMessages(library(stringr))
library(data.table)
library(parallel)

In [70]:
wd = '/nfs/lab/projects/lung_snATAC_tempJN/ciceroPooled'
zippedMatrixFile = '/nfs/lab/projects/lung_snATAC_tempJN/lung.merged_peaks.long_fmt.mtx.gz'
clusterFile = '/nfs/lab/projects/lung_snATAC_tempJN/lung_snATAC.merged_samples.cluster_labels_AllOneCluster.txt'
peakBedFile = '/nfs/lab/projects/lung_snATAC_tempJN/lung.merged_peaks.anno.ALL.bed'
outConnectionFilePrefix = 'lung_snATAC_new.'
outRdsConnsSuffix = '.1MB_cicero_conns.rds'
outConnsSuffix = '.cicero_conns.txt'
outConnsDedupSuffix = '.cicero_conns_dedup.txt'

In [71]:
clus <- makeCluster(32)
#clusterExport(clus)

In [72]:
setwd(wd)

In [73]:
zz   = gzfile(zippedMatrixFile,'rt')   ### long for matrix peak x barcode
data = read.table(zz,header=T,  sep="\t")

In [74]:
head(data)
dim(data)

peak,barcode,value
<fct>,<fct>,<int>
10:100002566-100002957,D032_AATGGTATCGCCGGATCAAGTT,2
10:100002566-100002957,D032_ACACAGAACACCTCATAGTAGA,2
10:100002566-100002957,D032_AGAACCTATCCGTAAGGCTAAC,2
10:100002566-100002957,D032_AGAGGCAAAGTTGTTCCTCTGC,2
10:100002566-100002957,D032_AGGTCGACTTGTCCAGATTTCC,1
10:100002566-100002957,D032_ATTGGCATGCAGGCATGCAGAG,2


[1] 96731882        3

In [75]:
#head(data)
colnames(data)[colnames(data)=="cell"] <- "barcode"
### convert to sparse matrix peak x barcode
sc.data <- with(data, sparseMatrix(i=as.numeric(peak), j=as.numeric(barcode), 
                                   x=value, dimnames=list(levels(peak), levels(barcode))))
rownames(sc.data) <- paste0('chr', gsub('-','_', gsub(':','_',rownames(sc.data))))


In [76]:
head(sc.data)
dim(sc.data)

   [[ suppressing 34 column names 'D032_AACGAGAGCTAACCACCTAAAG', 'D032_AACGAGAGCTAACCATGCATGA', 'D032_AACGAGAGCTAACGTGATTTGG' ... ]]


6 x 40264 sparse Matrix of class "dgCMatrix"
                                                                              
chr1_100000160_100000558 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100003553_100003886 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100009882_100010299 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100023018_100023218 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100023992_100024303 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100024847_100025096 . . . . . . . . . . . . . . . . . . . . . . . . . . .
                                             
chr1_100000160_100000558 . . . . . . . ......
chr1_100003553_100003886 . . . . . . . ......
chr1_100009882_100010299 . . . . . . . ......
chr1_100023018_100023218 . . . . . . . ......
chr1_100023992_100024303 . . . . . . . ......
chr1_100024847_100025096 . . . . . . . ......

 .....suppressing 40230 columns in show(); maybe adjust 'options(max.print= *, 

[1] 314586  40264

In [115]:
sc.umap <- read.table( clusterFile, sep='\t', header=T, row.names=1) ### map barcode to cluster 

In [116]:
head(sc.umap)
dim(sc.umap)

,UMAP1,UMAP2,subset2,cluster_name,log_usable_counts
,<dbl>,<dbl>,<fct>,<fct>,<dbl>
D062_AACGAGAGCTAAACCTAAGTGG,-3.965895,1.0771579,16,all,6.184149
D062_AACGAGAGCTAAACTAGCCCTA,2.340209,14.4901056,"4,0",all,8.326275
D062_AACGAGAGCTAAAGGAACAGAC,7.979088,1.6915314,3,all,6.549651
D062_AACGAGAGCTAAAGGATCGGCT,3.531774,14.2219507,"4,0",all,7.875879
D062_AACGAGAGCTAAGTGTCGTGGA,-11.856324,-0.8003238,0,all,7.232733
D062_AACGAGAGCTAATCCCGCGCTT,6.858509,2.9069133,3,all,6.364751


[1] 90980     5

In [43]:
cluster_peaks = read.table(peakBedFile, header=F) ### peaks called in each cluster to convert as wide format

In [79]:
head(cluster_peaks)
dim(cluster_peaks)

V1,V2,V3,V4
<fct>,<int>,<int>,<fct>
chr1,9991,10670,all
chr1,237643,237872,all
chr1,521477,521696,all
chr1,713683,714664,all
chr1,740184,740424,all
chr1,752356,752977,all


[1] 314586      4

In [ ]:



#uniqueClusters <-  unique(as.character(sc.umap$cluster))
#mat   = str_split_fixed( cluster_peaks[,4], "\\,", length(unique(sc.umap$cluster)))
#mat2  = t( apply( mat, 1, function(x) as.numeric(unique(sc.umap$cluster) %in% x)))
#peaks = cbind(paste(cluster_peaks[,1], cluster_peaks[,2],  cluster_peaks[,3], sep="_"), mat2)

#peaks = read.table('/nfs/lab/projects/lung_snATAC_tempJN/lung.merged_peaks.anno.ALL.clusterMatrix.txt', header=T)




#colnames(peaks) = c("peak", uniqueClusters)
#head(peaks)
#clusters <- unique(sc.umap$cluster)

In [118]:
sc.umap.subset <- sc.umap
sc.data.subset <- sc.data.subset[ ,colnames(sc.data.subset) %in% rownames(sc.umap.subset)] > 0

In [120]:
#for (cluster in clusters) {
   # print("subset umap and data")
   # sc.umap.subset <- sc.umap[sc.umap[['cluster']]==cluster,]
    #sc.data.subset <- sc.data[ rownames(sc.data) %in% peaks[peaks[, cluster]>0,'peak'], ]
    #head(sc.data.subset)
    sc.data.subset <- sc.data
    cellinfo <-data.frame(cells=colnames(sc.data.subset))
    row.names(cellinfo) <- cellinfo$cells
    dhsinfo <- data.frame(site_name=rownames(sc.data.subset))
    row.names(dhsinfo) <- dhsinfo$site_name
    dhsinfo <- cbind(dhsinfo, stringr::str_split_fixed(dhsinfo$site_name, "_", 3))
    names(dhsinfo) <- c('site_name','chr','bp1','bp2')
    dhsinfo$chr <- gsub('chr','', dhsinfo$chr)
    dhsinfo$bp1 <- as.numeric(as.character(dhsinfo$bp1))
    dhsinfo$bp2 <- as.numeric(as.character(dhsinfo$bp2))
    
    print("make input cds")
    input_cds <- suppressWarnings(newCellDataSet(as(sc.data.subset, 'dgCMatrix'),
                                phenoData = methods::new('AnnotatedDataFrame', data = cellinfo),
                                featureData = methods::new('AnnotatedDataFrame', data = dhsinfo),
                                expressionFamily=negbinomial.size(),
                                lowerDetectionLimit=0))
    
    
    print(head(fData(input_cds)))
    
    
    input_cds@expressionFamily <- binomialff()
    input_cds@expressionFamily@vfamily <- 'binomialff'
    input_cds <- detectGenes(input_cds)
    input_cds <- estimateSizeFactors(input_cds)
    input_cds <- input_cds[fData(input_cds)$num_cells_expressed > 0,]

        
     head(fData(input_cds))
    
    
    umap_coords <- sc.umap[colnames(sc.data.subset), c('UMAP1','UMAP2')]

    head(umap_coords)


    colnames(umap_coords) <- NULL




    cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = umap_coords, k=30)
    window <- 1e6
    data('human.hg19.genome')
    distance_parameters <- estimate_distance_parameter(cicero_cds, window=window, maxit=100, sample_num=100, distance_constraint=500000, genomic_coords=human.hg19.genome)
    mean_distance_parameter <- mean(unlist(distance_parameters))
    cicero_out <- generate_cicero_models(cicero_cds, distance_parameter=mean_distance_parameter, window=window, genomic_coords=human.hg19.genome)
    conns <- assemble_connections(cicero_out, silent=FALSE)
    saveRDS(conns, file.path(wd, paste0(outConnectionFilePrefix, cluster, outRdsConnsSuffix)))
    write.table(conns, file.path(wd, paste0(outConnectionFilePrefix, cluster, outConnsSuffix)), sep='\t', quote=FALSE, row.names=FALSE)

    ## this step is to remove duplicated connections
    conns = conns[order(-conns$coaccess),]
    bed = cbind(str_split_fixed(conns[,1], "\\_", 3 ), str_split_fixed(conns[,2], "\\_", 3 ))
    ord = matrix(parRapply(clus, bed, function(x) x[order(x)] ), ncol=6, byrow=T)
    ord = cbind(ord[, c(5,1:2,5,3:4)], conns$coaccess)
    dedup = ord[!duplicated(ord),]
    dedup = data.frame( Peak1 = paste(dedup[,1], dedup[,2], dedup[,3], sep="_")  , 
                        Peak2 = paste(dedup[,4], dedup[,5], dedup[,6], sep="_") , coaccess = dedup[,7]  )

              
  write.table(dedup, file.path(wd, paste0(outConnectionFilePrefix, cluster, outConnsDedupSuffix)), sep='\t', quote=FALSE, row.names=FALSE)
             
              
#}

[1] "make input cds"
                                        site_name chr       bp1       bp2
chr1_100000160_100000558 chr1_100000160_100000558   1 100000160 100000558
chr1_100003553_100003886 chr1_100003553_100003886   1 100003553 100003886
chr1_100009882_100010299 chr1_100009882_100010299   1 100009882 100010299
chr1_100023018_100023218 chr1_100023018_100023218   1 100023018 100023218
chr1_100023992_100024303 chr1_100023992_100024303   1 100023992 100024303
chr1_100024847_100025096 chr1_100024847_100025096   1 100024847 100025096


,site_name,chr,bp1,bp2,num_cells_expressed
,<fct>,<chr>,<dbl>,<dbl>,<int>
chr1_100000160_100000558,chr1_100000160_100000558,1,100000160,100000558,72
chr1_100003553_100003886,chr1_100003553_100003886,1,100003553,100003886,60
chr1_100009882_100010299,chr1_100009882_100010299,1,100009882,100010299,53
chr1_100023018_100023218,chr1_100023018_100023218,1,100023018,100023218,27
chr1_100023992_100024303,chr1_100023992_100024303,1,100023992,100024303,28
chr1_100024847_100025096,chr1_100024847_100025096,1,100024847,100025096,20


,UMAP1,UMAP2
,<dbl>,<dbl>
D032_AACGAGAGCTAACCACCTAAAG,5.466162,3.883028
D032_AACGAGAGCTAACCATGCATGA,-12.578452,-2.374089
D032_AACGAGAGCTAACGTGATTTGG,4.399629,4.022728
D032_AACGAGAGCTAAGCTCAACGCA,7.654104,3.517541
D032_AACGAGAGCTACCACGCAAACT,8.069730,2.221157
D032_AACGAGAGCTACGCCGTGACTT,2.107867,14.910593


ERROR: Error: setequal(x = row.names(reduced_coordinates), y = colnames(cds)) is not TRUE


In [ ]:
head(colnames())

In [122]:
sum(colnames(input_cds) %in% rownames(umap_coords))

[1] 36381

In [ ]:
head(fData(input_cds)[,1:10])

In [127]:
dim(umap_coords)

[1] 40264     2

In [128]:
sum(!duplicated(colnames(input_cds)))

[1] 40264